# Log likelihood calculation

$L(\theta) = -2\theta T + \int_0^T ((\alpha_s)_+ - (\alpha_s)_-)ds + \sum_{i=1}^{m+} log [(\alpha_{t_i}^{+ -})_+ + \theta] + \sum_{i=1}^{m+} log [(\alpha_{t_i}^{- -})_- + \theta]$

$\alpha_{t^-} = \eta^+ \sum_{i=1}^{n+}e^{-k(t-\tau_i^{0+})}1{t>\tau_i^{0+}} - \sum_{i=1}^{n-}e^{-k(t-\tau_i^{0-})}1{t>\tau_i^{0-}}$

https://docs.scipy.org/doc/scipy/tutorial/optimize.html

In [2]:
import numpy as np
from scipy.optimize import minimize
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])  # the 4 parameters to optimize
res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571


In [3]:
res


       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 4.861153433422115e-17
             x: [ 1.000e+00  1.000e+00  1.000e+00  1.000e+00  1.000e+00]
           nit: 339
          nfev: 571
 final_simplex: (array([[ 1.000e+00,  1.000e+00, ...,  1.000e+00,
                         1.000e+00],
                       [ 1.000e+00,  1.000e+00, ...,  1.000e+00,
                         1.000e+00],
                       ...,
                       [ 1.000e+00,  1.000e+00, ...,  1.000e+00,
                         1.000e+00],
                       [ 1.000e+00,  1.000e+00, ...,  1.000e+00,
                         1.000e+00]]), array([ 4.861e-17,  7.652e-17,  8.114e-17,  8.633e-17,
                        8.641e-17,  2.179e-16]))

In [3]:
import numpy as np
from scipy.optimize import minimize
from likelihood import MaximumLikelihood

T = 0.001
tau_0_plus = np.array([[0.1, 0.3, 0.5, 0.7, 0.9]]) * T
tau_0_minus = np.array([[0.2, 0.4, 0.6]]) * T
t_plus = np.array([[0.11, 0.31, 0.51, 0.71, 0.91]]) * T
t_minus = np.array([[0.21, 0.41, 0.61]]) * T

m = MaximumLikelihood(T, tau_0_plus, tau_0_minus, t_plus, t_minus)

x0 = np.array([1.3, 0.7, 0.8, 1.9])  # the 4 parameters to optimize
res = minimize(m.likelihood_to_minimize, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
res

/var/folders/rr/nsd1p9hx6hs_81m8swcypvkc0000gn/T/ipykernel_92448/1025083622.py:14: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  res = minimize(m.likelihood_to_minimize, x0, method='nelder-mead',


       message: Maximum number of function evaluations has been exceeded.
       success: False
        status: 1
           fun: -9.762244064601471e+24
             x: [ 1.288e+02  7.611e+27 -1.485e+28  1.925e+28]
           nit: 476
          nfev: 800
 final_simplex: (array([[ 1.288e+02,  7.611e+27, -1.485e+28,  1.925e+28],
                       [ 1.471e+02,  5.337e+27, -1.041e+28,  1.350e+28],
                       ...,
                       [ 1.745e+02,  3.856e+27, -7.524e+27,  9.753e+27],
                       [ 1.874e+02,  2.957e+27, -5.770e+27,  7.479e+27]]), array([-9.762e+24, -6.637e+24, -5.238e+24, -4.572e+24,
                       -3.426e+24]))

## Calculating integral of alpha_s

In [5]:
T=10
tau_0_plus = np.array([[0.1, 0.3, 0.5,0.7,0.9]]) * T
tau_0_minus = np.array([[0.2, 0.4, 0.6]])*T
tau_0 = np.concatenate(
    [np.zeros([1,1]), tau_0_minus, tau_0_plus, np.ones([1,1]) * T], axis=1
)

eta_plus = 100
eta_minus = 90
k = 200
eta_minus_vector = -np.ones([tau_0_minus.shape[1], 1]) * eta_minus
eta_plus_vector = np.ones([tau_0_plus.shape[1], 1]) * eta_plus
eta_vector = np.concatenate(
    [np.zeros([1,1]), eta_minus_vector, eta_plus_vector, np.zeros([1,1])], axis=0
).T
tau_eta = np.concatenate([tau_0, eta_vector])
result = eta_vector.T * np.ones([1, tau_0.shape[1]])

#tau_0.sort()
tau_eta = tau_eta[:,tau_eta[0, :].argsort()]
tau_0 = tau_eta[:,tau_eta[0, :].argsort()][0:1,:]
eta_0 = tau_eta[:,tau_eta[0, :].argsort()][1:2,:]
#print(result.T)
print(tau_eta)


[[  0.   1.   2.   3.   4.   5.   6.   7.   9.  10.]
 [  0. 100. -90. 100. -90. 100. -90. 100. 100.   0.]]


In [6]:
np.exp(-k*tau_0_plus)
tau_matrix = tau_0 * np.ones([1, tau_0.shape[1]]).T
tau_matrix

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.]])

In [7]:
np.exp(-k*tau_0_plus)
eta_matrix = eta_0 * np.ones([1, eta_0.shape[1]]).T
eta_matrix

array([[  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.]])

In [8]:
tau_matrix_1 = np.roll(tau_matrix,-1) # numero de fila es j, numero de columna es i
tau_matrix_1

array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.]])

In [9]:
tau_matrix_diff = tau_matrix - tau_matrix.T
tau_matrix_diff

array([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   9.,  10.],
       [ -1.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   8.,   9.],
       [ -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   5.,   7.,   8.],
       [ -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   6.,   7.],
       [ -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   5.,   6.],
       [ -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   4.,   5.],
       [ -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   3.,   4.],
       [ -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   2.,   3.],
       [ -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,   0.,   1.],
       [-10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -1.,   0.]])

In [10]:
tau_matrix_diff

array([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   9.,  10.],
       [ -1.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   8.,   9.],
       [ -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   5.,   7.,   8.],
       [ -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   6.,   7.],
       [ -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   5.,   6.],
       [ -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   4.,   5.],
       [ -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   3.,   4.],
       [ -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   2.,   3.],
       [ -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,   0.,   1.],
       [-10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -1.,   0.]])

In [11]:
tau_matrix_diff = np.where(tau_matrix_diff>0, tau_matrix_diff, 0)
tau_matrix_diff

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  8.,  9.],
       [ 0.,  0.,  0.,  1.,  2.,  3.,  4.,  5.,  7.,  8.],
       [ 0.,  0.,  0.,  0.,  1.,  2.,  3.,  4.,  6.,  7.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  2.,  3.,  5.,  6.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  4.,  5.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  3.,  4.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  3.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [12]:
arr = np.array([[1,2,3,3],[4,5,6,6]])
print(arr)
np.sum(arr, axis=0)

[[1 2 3 3]
 [4 5 6 6]]


array([5, 7, 9, 9])

In [13]:
tau_matrix_diff_1 = tau_matrix_1 - tau_matrix.T
tau_matrix_diff_1 = np.where(tau_matrix_diff_1>0, tau_matrix_diff_1, 0)
tau_matrix_diff_1


array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  8.,  9.,  0.],
       [ 0.,  0.,  1.,  2.,  3.,  4.,  5.,  7.,  8.,  0.],
       [ 0.,  0.,  0.,  1.,  2.,  3.,  4.,  6.,  7.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  2.,  3.,  5.,  6.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  2.,  4.,  5.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  3.,  4.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  3.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [14]:
tau_matrix_diff = tau_matrix - tau_matrix.T
tau_matrix_diff = np.where(tau_matrix_diff>0, tau_matrix_diff, 0)

tau_matrix_diff_1 = tau_matrix_1 - tau_matrix.T
tau_matrix_diff_1 = np.where(tau_matrix_diff_1>0, tau_matrix_diff_1, 0)

alpha_tau_matrix = eta_matrix * (np.exp(
    -k * tau_matrix_diff_1
) - np.exp(-k * tau_matrix_diff))

alpha_tau_matrix

array([[-0.00000000e+000, -1.38389653e-085,  1.72365264e-172,
        -2.65039655e-259, -0.00000000e+000,  0.00000000e+000,
        -0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000],
       [ 0.00000000e+000, -1.00000000e+002,  1.24550687e-085,
        -1.91516960e-172,  2.38535690e-259,  0.00000000e+000,
        -0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000],
       [ 0.00000000e+000,  0.00000000e+000,  9.00000000e+001,
        -1.38389653e-085,  1.72365264e-172, -2.65039655e-259,
        -0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000],
       [ 0.00000000e+000,  0.00000000e+000, -0.00000000e+000,
        -1.00000000e+002,  1.24550687e-085, -1.91516960e-172,
         2.38535690e-259,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000],
       [ 0.00000000e+000,  0.00000000e+000, -0.00000000e+000,
         0.00000000e+000,  9.00000000e+001, -1.38389653e-085,
         1.72365264e-172

In [15]:
alpha_tau = np.sum(alpha_tau_matrix, axis=0)
alpha_tau

array([   0., -100.,   90., -100.,   90., -100.,   90., -100., -100.,
          0.])

Alpha tau should be possitive. I don't know why are they using - 1/k. May be I should modifty it to be possitive inverting the simbols

In [16]:
alpha_s_plus = np.sum(-np.where(alpha_tau>=0,alpha_tau, 0)/k)
alpha_s_plus 

-1.35

In [17]:
alpha_s_minus = np.sum(np.where(alpha_tau<=0,alpha_tau, 0)/k)
alpha_s_minus

-2.5

In [18]:
integral_alpha_s = alpha_s_plus - alpha_s_minus
integral_alpha_s


1.15

## Calculating sum log alpha t_i

In [25]:
import numpy as np
T=10
tau_0_plus = np.array([[0.1, 0.3, 0.5,0.7,0.9]]) * T
tau_0_minus = np.array([[0.2, 0.4, 0.6]])*T
t_plus = np.array([[0.11, 0.31, 0.51, 0.71, 0.91]]) * T
t_minus = np.array([[0.21, 0.41, 0.61]]) * T
eta_plus = 100
eta_minus = 90
k = 200
theta = 1

tau_0 = np.concatenate(
    [
        tau_0_minus,
        tau_0_plus,
    ],
    axis=1,
)

eta_minus_vector = -np.ones([tau_0_minus.shape[1], 1]) * eta_minus
eta_plus_vector = np.ones([tau_0_plus.shape[1], 1]) * eta_plus
eta_vector = np.concatenate([eta_minus_vector, eta_plus_vector], axis=0).T
tau_eta = np.concatenate([tau_0, eta_vector])
tau_eta = tau_eta[:, tau_eta[0, :].argsort()]
tau_0 = tau_eta[:, tau_eta[0, :].argsort()][0:1, :]
eta_0 = tau_eta[:, tau_eta[0, :].argsort()][1:2, :]

tau_matrix = tau_0 * np.ones([1, t_plus.shape[1]]).T
eta_matrix = eta_0 * np.ones([1, t_plus.shape[1]]).T
t_plus_matrix = t_plus.T * np.ones([1, tau_0.shape[1]])  # numero de fila es t, numero de columna es tau

tau_matrix_diff = t_plus_matrix - tau_matrix 
tau_matrix_diff = np.where(tau_matrix_diff > 0, tau_matrix_diff, 0)

alpha_tau_matrix = eta_matrix * (np.exp(-k * tau_matrix_diff))
alpha_tau = np.sum(alpha_tau_matrix, axis=1)
# The signs seem to be wrong in the paper, keeping them anyway
sum_log_alpha_plus = np.sum(np.log(np.where(alpha_tau >= 0, alpha_tau, 0) + theta))
sum_log_alpha_plus

18.995638800241302

18.995638800241302

In [22]:
np.log(1)

0.0

array([1.30000000e+02, 1.20000000e+02, 1.10000000e+02, 1.00000000e+02,
       2.06115362e-07])

## Generating simulation data to estimate parameters


In [2]:
from simulations import generate_simulations_bunchs
from parameters import Parameters, base_simulation_parameters_dict
import pandas as pd
import numpy as np


p = Parameters(**base_simulation_parameters_dict)
p.dalpha = 30
print(p)
results = generate_simulations_bunchs(p)


Parameters(q_max=4, T=60, A=300, dalpha=30, Delta=0.005, epsilon=0.005, psi=0.01, phi_=1e-06, eta_plus=60.0, eta_minus=60.0, sigma=0.01, k=200.0, xi=1.0, lambda_plus=1.0, lambda_minus=1.0, theta=0.1, s0=100, n=200, drift=True)
Calculating h
Starting h calculation
0.0004995003608890108
120120
Starting to calculate h
Finished calculating h0.0 h                          3e-07 h                           
Amount of simulations: 200
Mean of PNL:0.2342749999998560285199e-07 h                              
Stde of PNL:0.18778471549898396
500


In [3]:
results["dMt_minus"]
results["dMt_plus"]
results["s"]
p.dt


0.0004995003608890108

In [23]:
tau_plus = (np.array(np.where(results["m_executions_count"][1,:-1]))*p.dt)[0]
tau_minus = (np.array(np.where(results["p_executions_count"][1,:-1]))*p.dt)[0]

print("Tau minus ", tau_minus)
print("Tau plus ", tau_plus)

Tau minus  [ 2.94255663  3.35564342  4.47452423  5.537461    8.70928829  8.91508244
  9.15983762  9.85813912 10.44205504 10.70079623 10.80469231 10.88011686
 11.50249431 11.64635041 11.91707961 13.9750211  15.87062497 16.28121426
 16.60039499 17.93555946 19.10938531 20.01997446 21.53495906 21.63086313
 22.05094293 25.7457471  29.26822365 29.79269903 31.35014115 31.49699426
 32.17281824 32.29269833 33.93455602 34.16432618 35.91007995 36.86012963
 37.31017946 37.65783171 38.01447497 38.72326598 40.82316549 41.00848013
 41.37561289 43.21527272 47.57041637 48.08240424 48.80618026 51.19179399
 55.39608852 55.9225619  56.75822601 57.31566841]
Tau plus  [ 0.19730264  0.81618359  1.72677275  2.19730209  3.22327583  3.94455435
  6.57442375  6.67432382  8.91757994  9.86113612 10.05344376 12.25873786
 13.142354   14.16732874 15.0799159  16.16832718 16.66133404 19.25973492
 19.96153292 20.33465969 21.42107298 24.31667657 27.37811428 27.4035888
 29.44904278 29.56043136 29.8975941  31.76622495 32.68

In [20]:
s_jumps = results['s']-np.roll(results['s'],1)
t_plus = (np.array(np.where(s_jumps>0)) * p.dt)[1]
t_minus = (np.array(np.where(s_jumps<0)) * p.dt)[1]
print("t_plus", t_plus)
print("t_minus", t_minus)
print("s_jumps", s_jumps)

t_plus [ 0.         10.31218495 18.38011478 ... 43.7367511  44.38760007
 46.68380323]
t_minus [ 1.09190779  1.85814134  6.91308499 ... 53.6967883  54.23774719
 56.78270153]
s_jumps [[ 0.06  0.    0.   ...  0.    0.    0.  ]
 [ 0.14  0.    0.   ...  0.    0.    0.  ]
 [ 0.12  0.    0.   ...  0.    0.    0.  ]
 ...
 [-0.08  0.    0.   ...  0.    0.    0.  ]
 [ 0.03  0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.  ]]


In [ ]:
import numpy as np
from scipy.optimize import minimize
from likelihood import MaximumLikelihood

tau_0_plus = tau_plus
tau_0_minus = tau_minus

m = MaximumLikelihood(
    p.T,
    tau_plus[0,:],
    tau_minus[0,:],
    t_plus,
    t_minus
)

x0 = np.array([1.3, 0.7, 0.8, 1.9])  # the 4 parameters to optimize
res = minimize(m.likelihood_to_minimize, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})